### ■ Overview

I thought a simple and easy post-processing based on my experience of using google maps on a daily basis.  
Have you ever had an experience where the google map location information was way off?  I thought that the data set of this competition might contain such outliers.


In this notebook, Correct the outliers in the baseline and apply the Kalman filter afterwards.

Score(Public Leaderboard)  
Original : 7.190  
outlier correction : 7.180  
Kalman filter : 6.164  

### ■ Reference
I used these great notebooks as a guide in creating this notebook. Thank you for publishing the notebook.


(Baseline :by JohnM)
https://www.kaggle.com/jpmiller/baseline-from-host-data

(Kalman filter : by Marcin Bodych)
https://www.kaggle.com/emaerthin/demonstration-of-the-kalman-filter

### ■ Update (2021-06-02)
I got some advice from @Kyosuke0924 .I changed some code to speed up the process.

### Step.1 Check for outliers in training data --------------------------------------------

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
data_path = Path("../input/google-smartphone-decimeter-challenge")
df_test = pd.read_csv(
    data_path / 'baseline_locations_test.csv')
df_sub    = pd.read_csv(
    data_path / 'sample_submission.csv')

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
truths = (data_path / 'train').rglob('ground_truth.csv')
    # returns a generator

df_list = []
cols = ['collectionName', 'phoneName', 'millisSinceGpsEpoch', 'latDeg',
       'lngDeg']

for t in tqdm(truths, total=73):
    df_phone = pd.read_csv(t, usecols=cols)  
    df_list.append(df_phone)
df_truth = pd.concat(df_list, ignore_index=True)

df_basepreds = pd.read_csv(data_path / 'baseline_locations_train.csv', usecols=cols)
df_all = df_truth.merge(df_basepreds, how='inner', on=cols[:3], suffixes=('_truth', '_basepred'))

In [ ]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [ ]:
df_all['dist'] = calc_haversine(df_all.latDeg_truth, df_all.lngDeg_truth, 
    df_all.latDeg_basepred, df_all.lngDeg_basepred)

In [ ]:
df_all.dist.describe()

In [ ]:
df_all.sort_values(by = 'dist',ascending = False)[['collectionName','dist']].head(10)

There are some points where the difference between baseline and ground truth is more than 1,000 meters.It is surprising.  
Dealing with these outliers is likely to improve the score.

### Step.2 Correct outliers in the test data.----------------------------------------------

Since there is no 'dist' in the test data, identify outliers based on their distance from the previous and next pass.

In [ ]:
df_test['dist_pre'] = 0
df_test['dist_pro'] = 0

In [ ]:
df_test['latDeg_pre'] = df_test['latDeg'].shift(periods=1,fill_value=0)
df_test['lngDeg_pre'] = df_test['lngDeg'].shift(periods=1,fill_value=0)
df_test['latDeg_pro'] = df_test['latDeg'].shift(periods=-1,fill_value=0)
df_test['lngDeg_pro'] = df_test['lngDeg'].shift(periods=-1,fill_value=0)
df_test['dist_pre'] = calc_haversine(df_test.latDeg_pre, df_test.lngDeg_pre, df_test.latDeg, df_test.lngDeg)
df_test['dist_pro'] = calc_haversine(df_test.latDeg, df_test.lngDeg, df_test.latDeg_pro, df_test.lngDeg_pro)

list_phone = df_test['phone'].unique()
for phone in list_phone:
    ind_s = df_test[df_test['phone'] == phone].index[0]
    ind_e = df_test[df_test['phone'] == phone].index[-1]
    df_test.loc[ind_s,'dist_pre'] = 0
    df_test.loc[ind_e,'dist_pro'] = 0

In [ ]:
df_test.dist_pre.describe()

There seems to be an outlier, although it is smaller than the training data.

If the distance between the previous and next pass is large, it is either an outlier or a case of speed (straight line movement), so fix the value to the middle of the previous and next pass.

In [ ]:
pro_95 = df_test['dist_pro'].mean() + (df_test['dist_pro'].std() * 2)
pre_95 = df_test['dist_pre'].mean() + (df_test['dist_pre'].std() * 2)
ind = df_test[(df_test['dist_pro'] > pro_95)&(df_test['dist_pre'] > pre_95)][['dist_pre','dist_pro']].index

for i in ind:
    df_test.loc[i,'latDeg'] = (df_test.loc[i-1,'latDeg'] + df_test.loc[i+1,'latDeg'])/2
    df_test.loc[i,'lngDeg'] = (df_test.loc[i-1,'lngDeg'] + df_test.loc[i+1,'lngDeg'])/2

### Step.3 Kalman filter -----------------------------------------------------------------

Finally, apply the Kalman filter.

In [ ]:
!pip install simdkalman

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import simdkalman
from tqdm.notebook import tqdm

In [ ]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

In [ ]:
def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in tqdm(unique_paths):
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [ ]:
kf_smoothed_baseline = apply_kf_smoothing(df_test)
df_sub = df_sub.assign(
    latDeg = kf_smoothed_baseline.latDeg,
    lngDeg = kf_smoothed_baseline.lngDeg
)
df_sub.to_csv('submission.csv', index=False)

#### Thank you for reading to the end.I look forward to your comments.